Notebook uses BedMachine Antarctica ice-surface and bed topographies to calculate and export subglacial hydropotential using the Shreve (1972) hydropotential equation.

Written 2025/05/19 by W. Sauthoff (wsauthoff.github.io) and M. R. Siegfried (mrsiegfried.github.io).

# Setup computing environment

In [ ]:
# Import packages
import earthaccess
import fsspec
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
import shutil
import xarray as xr
import zarr

%matplotlib widget

# Access data

In [ ]:
# Direct-access stream BedMachine Antarctica bed and surface topography v3 (Morlighem et al., 2020; Morlighem, 2022)
# https://nsidc.org/data/NSIDC-0756
try:
    results = earthaccess.search_data(
        doi='10.5067/FPSU0V1MWUB6',
        cloud_hosted=True,
        # bounding_box=(1, -89, -1, -89)
    )
    
    if not results:
        raise ValueError('No datasets found matching the search criteria')
        
    # Open data granules as s3 files to stream
    files = earthaccess.open(results)
    
    # Check if files list is empty
    if not files:
        raise FileNotFoundError('No files were opened from the search results')
        
    # Check if first file exists/is valid
    if isinstance(files[0], FileNotFoundError):
        raise FileNotFoundError(f'Could not access file: {files[0]}')
        
    # Print file name to ensure expected dataset
    print(f'Attempting to open: {files[0]}')
    
    # Open file into xarray dataset
    bedmachine_original = xr.open_dataset(files[0], engine='netcdf4')
    print('Dataset successfully loaded')

except FileNotFoundError as e:
    print(f'File access error: {e}')
except ValueError as e:
    print(f'Search error: {e}')
except Exception as e:
    print(f'Unexpected error: {e}')

'''
Note: Search error: can only read bytes or file-like objects with engine='scipy' or 'h5netcdf' seems to indicate FileNotFoundError
'''

In [ ]:
# Open local copy of file if streaming fails
bedmachine_original = xr.open_dataset('/home/jovyan/temp/BedMachineAntarctica-v3.nc')
bedmachine_original

# Data analysis
Use Shreve (1972) equation to calculate hydropotential using bed elevations and ice-surface elevations (less firn depth to get ice mass).

In [ ]:
# Step 1: Compute hydropotential

# Use BedMachine bed topography for Zbed and surface topography (less firn) for Zsurf

# Define densities (rho) of ice and water
rho_ice = 917
rho_water = 997

# Calculate hydropotential
subglacial_hydropotential_kPa = (9.8 * ((rho_ice*(bedmachine_original['surface']-bedmachine_original['firn'])) + (rho_water-rho_ice)*bedmachine_original['bed'])) / 1e3

# Display xarray.Dataset metadata
subglacial_hydropotential_kPa

In [ ]:
# Plot results
plt.figure()
subglacial_hydropotential_kPa.plot()
plt.show()

In [ ]:
# Step 2: Mask to grounded ice only

# Create a mask where values == 2 (grounded ice)
grounded_ice_mask = (bedmachine_original['mask'] == 2)

# Apply the mask to subglacial_hydropotential_kPa
subglacial_hydropotential_kPa = subglacial_hydropotential_kPa.where(grounded_ice_mask)

# Display xarray.Dataset metadata to ensure edges have become nan's
subglacial_hydropotential_kPa

# Export hydropotental dataset

In [ ]:
# Add metadata
subglacial_hydropotential_kPa.attrs = {
    'long_name': 'hydropotential',
    'units': 'kPa',
    'ice_density' : 917.0,
    'ice_density_units': 'kg m-3',
    'freshwater_density' : 997.0,
    'freshwater_density_units': 'kg m-3',
    'references': 'dois: 10.3189/S002214300002219X, 10.1038/s41561-019-0510-8, 10.5067/FPSU0V1MWUB6'
}

# Convert to dataset for chucking with Zarr
ds = subglacial_hydropotential_kPa.to_dataset(name='hydropotential')

# Add metadata
ds.attrs = {
    'conventions': 'CF-1.8',
    'title': 'Antarctic subglacial hydropotential derived from BedMachine Antarctica',
    'description': 'Antarctic subglacial gridded hydropotential calculated using Shreve (1972) equation using BedMachine Antarctica v3.4 bed and ice topographies and firn air content (Morlighem et al., 2020; Morlighem, 2022).',
    'history': 'Created 2025-05-19',
    # 'identifier_product_DOI': 'doi:',  # Awaiting first version release for Zenodo to mint DOI
    # 'citation': 'Sauthoff, W. and Siegfried, M. R. (2025). Antarctic subglacial hydropotential [Data set]. Zenodo. https://doi.org/XX',
    'license': 'CC BY-SA 4.0'
}

# Copy mapping variable into new dataset
ds['mapping'] = bedmachine_original['mapping']

# Link the mapping variable to your variable
ds['hydropotential'].attrs['grid_mapping'] = 'mapping'

# Copy global CRS-related attributes
ds.attrs['proj4'] = bedmachine_original.attrs['proj4']
ds.attrs['Projection'] = bedmachine_original.attrs['Projection']

# Add crs and crs_wkt
ds.attrs['crs'] = 'EPSG:3031'
crs = CRS.from_epsg(3031)
ds.attrs['crs_wkt'] = crs.to_wkt()

# View ds metadata
ds

## Export to chucked netcdf

In [ ]:
# Export with chunking + compression
ds.to_netcdf(
    '/home/jovyan/temp/Antarctic_hydropotential.nc',
    encoding={
        'hydropotential': {
            '_FillValue': -9999.0,
            'dtype': 'float32',
            'zlib': True,
            'complevel': 4,
            'chunksizes': (500, 500)
        }
    }
)

## Validate dataset after export

In [ ]:
conda install -c conda-forge cfchecker  --quiet

In [ ]:
!cfchecks /home/jovyan/temp/Antarctic_hydropotential.nc

In [ ]:
print(ds['mapping'].dims)
print(ds['mapping'].shape)
print(ds['mapping'].attrs)

In [ ]:
import numpy as np
ds = ds.drop_vars('mapping', errors='ignore')
mapping_attrs = bedmachine_original['mapping'].attrs
ds['mapping'] = xr.DataArray(np.array(0, dtype=np.int32), attrs=mapping_attrs)

In [ ]:
# Export with chunking + compression
ds.to_netcdf(
    '/home/jovyan/temp/Antarctic_hydropotential.nc',
    encoding={
        'hydropotential': {
            'zlib': True,
            'complevel': 4,
            'chunksizes': (500, 500)
        }
    }
)

In [ ]:
!cfchecks /home/jovyan/temp/Antarctic_hydropotential.nc

In [ ]:
conda install -c conda-forge netcdf4  --quiet

In [ ]:
!ncdump -h /home/jovyan/temp/Antarctic_hydropotential.nc

In [ ]:
# Opened in QGIS and Panoply to ensure compatibility

## Export to Zarr

In [ ]:
# Chunk for performance (tune based on size and use case)
ds = ds.chunk({'x': 500, 'y': 500})

# Write to a Zarr store
ds.to_zarr('/home/jovyan/temp/Antarctic_hydropotential.zarr', mode='w')

In [ ]:
# ds = xr.open_zarr('/home/jovyan/temp/Antarctic_hydropotential.zarr', consolidated=False)
# ds.to_zarr('/home/jovyan/temp/Antarctic_hydropotential.zarr', mode="w", consolidated=True)

In [ ]:
# Consolidate metadata before zipping
zarr.consolidate_metadata('/home/jovyan/temp/Antarctic_hydropotential.zarr')

# Zip zarr files for upload to Zenodo
shutil.make_archive(
    '/home/jovyan/temp/Antarctic_hydropotential.zarr',  # output path (no zip extension)
    'zip',
    '/home/jovyan/temp/Antarctic_hydropotential.zarr'   # source
)

## Validate dataset after export

In [ ]:
ds_imported = xr.open_zarr('/home/jovyan/temp/Antarctic_hydropotential.zarr', consolidated=True)
ds_imported

In [ ]:
ds_imported.info()

In [ ]:
plt.figure()
ds_imported['hydropotential'].plot(vmin=0, vmax=40e3)
plt.show()

In [ ]:
print(ds_imported.coords)
print(ds_imported.attrs.get("crs"))
print(ds_imported.attrs.get("proj4"))
print(ds_imported["mapping"].attrs)
print(ds_imported["hydropotential"].attrs)

In [ ]:
print(ds_imported.chunks)

In [ ]:
# Open the ZIP file to ensure data reads in as expected
with open("/home/jovyan/temp/Antarctic_hydropotential.zarr.zip", "rb") as f:
    # Use fsspec's zip filesystem with that file object
    fs = fsspec.filesystem("zip", fo=f)
    
    # Since the Zarr store is at the root of the zip, root path is just ""
    mapper = fs.get_mapper("")
    
    ds = xr.open_zarr(mapper, consolidated=True)

# View dataset metadata
ds

Exported and validated data file saved to Zenodo repository (Sauthoff & Siegfriend, 2025). 

# References

Morlighem, M., Rignot, E., Binder, T., Blankenship, D., Drews, R., Eagles, G., et al. (2020). Deep glacial troughs and stabilizing ridges unveiled beneath the margins of the Antarctic ice sheet. Nature Geoscience, 13(2), 132–137. https://doi.org/10.1038/s41561-019-0510-8

Morlighem, M. (2022). MEaSUREs BedMachine Antarctica. (NSIDC-0756, Version 3). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/FPSU0V1MWUB6. Date Accessed 05-19-2025.

Sauthoff, W., Siegfried, M. R. (2025). Antarctic subglacial hydropotential [Data set]. Zenodo. https://doi.org/XX

Shreve, R. L. (1972). Movement of Water in Glaciers. Journal of Glaciology, 11(62), 205–214. https://doi.org/10.3189/S002214300002219X